In [1]:
sc

In [4]:
!rm -rf metastore_db/*.lck

from pyspark.sql import SQLContext
sqlc = SQLContext(sc)

In [5]:
from pyspark.mllib.util import MLUtils

data = MLUtils.loadLibSVMFile(sc, "sample_libsvm_data.txt").toDF()
data = MLUtils.convertVectorColumnsToML(data)

In [6]:
data.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(692,[127,128,129...|  0.0|
|(692,[158,159,160...|  1.0|
|(692,[124,125,126...|  1.0|
|(692,[152,153,154...|  1.0|
|(692,[151,152,153...|  1.0|
+--------------------+-----+
only showing top 5 rows



## Evaluation

### Binary Classification

In [17]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import BinaryLogisticRegressionSummary

trainingData, testData = data.randomSplit([0.7, 0.3])

logr = LogisticRegression().setMaxIter(10).setRegParam(0.3).setElasticNetParam(0.8)

logrModel = logr.fit(trainingData)

print "Weights: %s Intercept: %s" % (logrModel.coefficients, logrModel.intercept)

Weights: (692,[235,243,244,262,263,271,272,290,300,301,323,328,329,350,351,358,378,379,385,386,401,405,406,407,413,427,428,429,433,434,435,455,456,461,462,469,483,484,489,490,496,497,511,512,517,539,540,568],[-1.08704076463e-05,-0.000225035191295,-1.42785721134e-06,-2.67838954428e-06,-0.000260749167911,-0.000108151886583,-0.000222047179992,-0.000184684690252,-0.000368205319861,-8.45317828463e-05,8.41700764597e-05,-8.99150674055e-06,-2.6018144065e-06,0.000106542521448,0.000214847611674,-8.02001292779e-06,0.000254341232325,0.000218251840905,-3.96099160439e-05,-2.89412079021e-05,-4.45357563079e-06,5.02740058996e-05,0.00084364056682,0.000260914756075,-2.13472989153e-05,-1.2101806874e-06,-0.000135806709886,-9.35571716832e-06,0.000231569631081,0.000943064886038,7.40416456292e-05,-1.3984003133e-05,-0.000254944102754,0.000131412325807,0.000884111338263,-9.88044455622e-05,-8.10057764206e-05,-0.000193685045759,0.00011911644026,0.000142456462059,-0.000123316509266,-8.01517008313e-05,-0.0003900993

In [12]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

predictionsLogR = logrModel.transform(testData)

evaluator = BinaryClassificationEvaluator().setLabelCol("label") \
                            .setRawPredictionCol("rawPrediction") \
                            .setMetricName("areaUnderROC")

roc = evaluator.evaluate(predictionsLogR)
print roc

1.0


### Multiclass Classification

In [18]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, IndexToString, VectorIndexer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import RandomForestClassificationModel

labelIndexer = StringIndexer().setInputCol("label") \
                                .setOutputCol("indexedLabel").fit(data)

labelConverter = IndexToString().setInputCol("prediction") \
                                .setOutputCol("predictedLabel") \
                                .setLabels(labelIndexer.labels)

featureIndexer = VectorIndexer().setInputCol("features") \
                                .setOutputCol("indexedFeatures") \
                                .setMaxCategories(4).fit(data)

rfC = RandomForestClassifier().setLabelCol("indexedLabel") \
                                .setFeaturesCol("indexedFeatures") \
                                .setNumTrees(3)
        
pipelineRFC = Pipeline().setStages([labelIndexer, featureIndexer, rfC, labelConverter])

modelRFC = pipelineRFC.fit(trainingData)

predictionsRFC = modelRFC.transform(testData)

In [19]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator().setLabelCol("indexedLabel") \
                                        .setPredictionCol("prediction") \
                                        .setMetricName("accuracy")

accuracy = evaluator.evaluate(predictionsRFC)

print "Test Error = %s" % (1.0 - accuracy)

Test Error = 0.0606060606061


### Regression

In [20]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.regression import RandomForestRegressionModel

rfR = RandomForestRegressor().setLabelCol("label").setFeaturesCol("indexedFeatures")

pipelineRFR = Pipeline().setStages([featureIndexer, rfR])

modelRFR = pipelineRFR.fit(trainingData)

predictionsRFR = modelRFR.transform(testData)

In [21]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator().setLabelCol("label") \
                                .setPredictionCol("prediction") \
                                .setMetricName("rmse")

rmse = evaluator.evaluate(predictionsRFR)

print "Root Mean Squared Error (RMSE) = %s" % rmse

Root Mean Squared Error (RMSE) = 0.138169855942


### Logistic Regression

In [23]:
from pyspark.ml.classification import LogisticRegression

logr = LogisticRegression().setMaxIter(10).setRegParam(0.3).setElasticNetParam(0.8)

logrModel = logr.fit(trainingData)

print "Weights: %s Intercept: %s" % (logrModel.coefficients, logrModel.intercept)

Weights: (692,[235,243,244,262,263,271,272,290,300,301,323,328,329,350,351,358,378,379,385,386,401,405,406,407,413,427,428,429,433,434,435,455,456,461,462,469,483,484,489,490,496,497,511,512,517,539,540,568],[-1.08704076463e-05,-0.000225035191295,-1.42785721134e-06,-2.67838954428e-06,-0.000260749167911,-0.000108151886583,-0.000222047179992,-0.000184684690252,-0.000368205319861,-8.45317828463e-05,8.41700764597e-05,-8.99150674055e-06,-2.6018144065e-06,0.000106542521448,0.000214847611674,-8.02001292779e-06,0.000254341232325,0.000218251840905,-3.96099160439e-05,-2.89412079021e-05,-4.45357563079e-06,5.02740058996e-05,0.00084364056682,0.000260914756075,-2.13472989153e-05,-1.2101806874e-06,-0.000135806709886,-9.35571716832e-06,0.000231569631081,0.000943064886038,7.40416456292e-05,-1.3984003133e-05,-0.000254944102754,0.000131412325807,0.000884111338263,-9.88044455622e-05,-8.10057764206e-05,-0.000193685045759,0.00011911644026,0.000142456462059,-0.000123316509266,-8.01517008313e-05,-0.0003900993

In [24]:
trainingSummaryLR = logrModel.summary
trainingSummaryLR.areaUnderROC

1.0

In [25]:
fMeasure = trainingSummaryLR.fMeasureByThreshold

fMeasure.show(3)

+------------------+-------------------+
|         threshold|          F-Measure|
+------------------+-------------------+
|0.8165428632958064|               0.05|
|0.8162694676199894|0.09756097560975609|
|0.8162646945004736|0.14285714285714288|
+------------------+-------------------+
only showing top 3 rows



In [26]:
from pyspark.sql import functions as F

maxFMeasure = fMeasure.agg({"F-Measure": "max"}).head()[0]
print maxFMeasure
maxFMeasure = fMeasure.agg(F.max(F.col("F-Measure"))).head()[0]
print maxFMeasure

bestThreshold = fMeasure.where(F.col("F-Measure") == maxFMeasure).select("threshold").head()[0]
print bestThreshold

1.0
1.0
0.613903946897


In [27]:
trainingSummaryLR.pr.show(3)
trainingSummaryLR.precisionByThreshold.show(3)

+-------------------+---------+
|             recall|precision|
+-------------------+---------+
|                0.0|      1.0|
|0.02564102564102564|      1.0|
|0.05128205128205128|      1.0|
+-------------------+---------+
only showing top 3 rows

+------------------+---------+
|         threshold|precision|
+------------------+---------+
|0.8165428632958064|      1.0|
|0.8162694676199894|      1.0|
|0.8162646945004736|      1.0|
+------------------+---------+
only showing top 3 rows



In [28]:
trainingSummaryLR.recallByThreshold.show(3)
trainingSummaryLR.roc.show(3)

+------------------+-------------------+
|         threshold|             recall|
+------------------+-------------------+
|0.8165428632958064|0.02564102564102564|
|0.8162694676199894|0.05128205128205128|
|0.8162646945004736|0.07692307692307693|
+------------------+-------------------+
only showing top 3 rows

+---+-------------------+
|FPR|                TPR|
+---+-------------------+
|0.0|                0.0|
|0.0|0.02564102564102564|
|0.0|0.05128205128205128|
+---+-------------------+
only showing top 3 rows



### Linear Regression

In [30]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression().setMaxIter(10).setRegParam(0.3).setElasticNetParam(0.8)

lrModel = lr.fit(trainingData)

print "Weights: %s Intercept: %s" % (lrModel.coefficients, lrModel.intercept)

Weights: (692,[351,378,379,406,407,433,434,462,490,517,540],[0.000133767304687,0.00018419522405,0.000135554811074,0.000275780406915,0.000159724075215,0.000144485199988,0.000306915977164,0.000288535540884,0.000120301061362,0.000123576266478,-0.00011823301358]) Intercept: 0.34681682126


In [31]:
trainingSummaryLLS = lrModel.summary

print trainingSummaryLLS.explainedVariance

print trainingSummaryLLS.meanAbsoluteError

print trainingSummaryLLS.meanSquaredError

print trainingSummaryLLS.r2

0.0506615309323
0.271181060072
0.0785812855913
0.676967590642


In [32]:
trainingSummaryLLS.residuals.show(3)

print trainingSummaryLLS.rootMeanSquaredError

+-------------------+
|          residuals|
+-------------------+
|0.24739337680570395|
|-0.3169038688245795|
| 0.5361913707012146|
+-------------------+
only showing top 3 rows

0.280323537348
